In [5]:
import pandas as pd
import os
from datetime import date
import numpy as np
import random

In [6]:
current_season = int(input('What is the current season?'))

In [7]:
if not os.getcwd().endswith('Football Forecasting Version 2'):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

dir_path = os.getcwd()

In [8]:
# CREATES DICTIONARY OF THREE LETTER ON CLUBS

res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'Matchweek' in path:
        res.append(path)

three_let_dict = {}
for matchweek_folder in res:
    os.chdir(fr'{dir_path}\{matchweek_folder}')
    a = pd.read_csv(random.choice(os.listdir(fr'{dir_path}\{matchweek_folder}')))
    for index, row in a.iterrows():
        three_let_dict[row['Club'].strip()] =  row['Code']
    os.chdir(dir_path)


three_let_dict['Tottenham'] = three_let_dict.pop('Tottenham Hotspur')
three_let_dict['Brighton'] = three_let_dict.pop('Brighton and Hove Albion')
three_let_dict['Manchester Utd'] = three_let_dict.pop('Manchester United')
three_let_dict['Newcastle Utd'] = three_let_dict.pop('Newcastle United')
three_let_dict['West Ham'] = three_let_dict.pop('West Ham United')
three_let_dict['Wolves'] = three_let_dict.pop('Wolverhampton Wanderers')
three_let_dict["Nott'ham Forest"] = three_let_dict.pop('Nottingham Forest')
three_let_dict["Sheffield Utd"] = three_let_dict.pop('Sheffield United')
three_let_dict["Huddersfield"] = three_let_dict.pop('Huddersfield Town')
three_let_dict["West Brom"] = three_let_dict.pop('West Bromwich Albion')

In [9]:
### FIXTURE LIST

res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'Fixture' in path:
        res.append(path)
print (res)
df = pd.DataFrame()
for fixtures in res:
    os.chdir(fr'{dir_path}\{fixtures}')
    folder = os.getcwd()
    list_of_files = os.listdir()
    for i in list_of_files:
        b = pd.read_csv(fr'{folder}\{i}', index_col = 0)
        b['Date'] = pd.to_datetime(b['Date'])
        b['Club'] = b['Club'].map(three_let_dict)
        b['Opponent'] = b['Opponent'].map(three_let_dict)
        b = b[b['Date'] > np.datetime64('today')].iloc[:1]
        df = pd.concat([df,b])
    os.chdir(dir_path)
df = df.reset_index(drop = True)
df = df.sort_values(by = ['Date'] )
df = df.reset_index(drop = True)





['2017_2018_Fixture', '2018_2019_Fixture', '2019_2020_Fixture', '2020_2021_Fixture', '2021_2022_Fixture', '2022_2023_Fixture']


In [10]:
### SPI DATA

res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'SPI' in path:
        res.append(path)
print (res)
df1 = pd.DataFrame()
for SPIdata in res:
    os.chdir(fr'{dir_path}\{SPIdata}')
    folder = os.getcwd()
    list_of_files = os.listdir()
    for i in list_of_files:
        b = pd.read_csv(fr'{folder}\{i}', index_col=0)
        b['date'] = pd.to_datetime(b['date'])
        b['Season'] = int(i[:4])
        b['team'] = b['team'].str.replace('Leicester', 'Leicester City')
        b['team'] = b['team'].str.replace('Newcastle', 'Newcastle Utd')
        b['team'] = b['team'].str.replace('Southamon', 'Southampton')
        b['team'] = b['team'].str.replace('Norwich', 'Norwich City')
        b['team'] = b['team'].str.replace('Man City', 'Manchester City')
        b['team'] = b['team'].str.replace('Man United', 'Manchester Utd')
        b['team'] = b['team'].str.replace('Nottm Forest', "Nott'ham Forest")
        b['team'] = b['team'].map(three_let_dict)
        df1 = pd.concat([df1,b])
    os.chdir(dir_path)
df1 = df1.reset_index(drop = True)
df1 = df1.sort_values(by = ['date'] )


['2017_2018_SPIdata', '2018_2019_SPIdata', '2019_2020_SPIdata', '2020_2021_SPIdata', '2021_2022_SPIdata', '2022_2023_SPIdata']


In [11]:
res = []
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a folder
    if os.path.isdir(os.path.join(dir_path, path)) and 'Matchweek' in path:
        res.append(path)

df2 = pd.DataFrame()
for matchweek in res:
    os.chdir(fr'{dir_path}\{matchweek}')
    folder = os.getcwd()
    list_of_files = os.listdir()
    for i in list_of_files:
            a = pd.read_csv(fr'{folder}\{i}', index_col = 0)
            a['Season'] = int(matchweek[:4])
            a['Last Match Date'] = pd.to_datetime(a['Last Match Date'])
            a = a.drop (columns = ['Club','W','D','L','GF', 'GA', 'GD'])
            df2 = pd.concat([df2,a])
    os.chdir(dir_path)

df2 = df2.sort_values(by = ['Last Match Date'])

In [12]:
fixture_fixture_merge  = pd.merge_asof(left=df,right=df,left_on = ['Date'], right_on = ['Date'], left_by = ['Club'], right_by = ['Opponent'], direction='nearest')
fixture_fixture_merge = fixture_fixture_merge.drop(columns = ['Venue_y', 'Result_y','Opponent_y','Club_y'])

In [13]:
matchweek_fixture_merge_1 = pd.merge_asof(left=fixture_fixture_merge ,right=df2,left_on = ['Date'], right_on = ['Last Match Date'], left_by = ['Club_x'], right_by = ['Code'], direction='nearest')
matchweek_fixture_merge_1 = matchweek_fixture_merge_1.drop (columns = ['Last Match Date', 'Code']) 
matchweek_fixture_merge_2 = pd.merge_asof(left=matchweek_fixture_merge_1,right=df2,left_on = ['Date'], right_on = ['Last Match Date'], left_by = ['Opponent_x'], right_by = ['Code'], direction='nearest')
matchweek_fixture_merge_2 = matchweek_fixture_merge_2.drop (columns = ['Season_x','Last Match Date', 'Code']) 

In [14]:
spi_everything_merge_1 = pd.merge_asof(left=matchweek_fixture_merge_2, right=df1, left_on = ['Date'], right_on = ['date'], left_by = ['Club_x'], right_by = ['team'], direction='nearest')
spi_everything_merge_1 = spi_everything_merge_1.drop (columns = ['team','date', 'Season_y']) 
spi_everything_merge_2 = pd.merge_asof(left=spi_everything_merge_1 , right=df1, left_on = ['Date'], right_on = ['date'], left_by = ['Opponent_x'], right_by = ['team'], direction='nearest')
spi_everything_merge_2 = spi_everything_merge_2.drop (columns = ['team','date', 'Season_y', 'GF_y', 'GA_y', 'xG_y', 'xGA_y']) 



In [15]:
rename_columns = {'Venue_x' : 'Venue', 'Result_x' : 'Result', 'Club_x':'Club', 'Season_x' : 'Season', 'Opponent_x': 'Opp'}

df_final = spi_everything_merge_2.rename(rename_columns, axis = 'columns')

display(df_final)

,Date,Venue,Result,GF_x,GA_x,Opp,xG_x,xGA_x,Poss_x,Club,...,Pl_y,Pts_y,Form_y,spi_x,off_x,def_x,Season,spi_y,off_y,def_y
0,2023-03-11,Home,NaN,NaN,NaN,BHA,NaN,NaN,NaN,LEE,...,23,38,1.6,67.3,1.8,0.9,2022,82.0,2.3,0.6
1,2023-03-11,Home,NaN,NaN,NaN,NFO,NaN,NaN,NaN,TOT,...,25,26,1.0,78.4,2.2,0.7,2022,55.4,1.5,1.1
2,2023-03-11,Home,NaN,NaN,NaN,LIV,NaN,NaN,NaN,BOU,...,25,42,2.6,57.1,1.6,1.2,2022,84.8,2.7,0.6
3,2023-03-11,Away,NaN,NaN,NaN,EVE,NaN,NaN,NaN,BRE,...,26,22,0.8,73.5,2.1,0.8,2022,61.1,1.7,1.0
4,2023-03-11,Away,NaN,NaN,NaN,LEE,NaN,NaN,NaN,BHA,...,25,22,0.8,82.0,2.3,0.6,2022,67.3,1.8,0.9
5,2023-03-11,Away,NaN,NaN,NaN,LEI,NaN,NaN,NaN,CHE,...,25,24,1.2,80.6,2.0,0.4,2022,69.2,1.9,0.9
6,2023-03-11,Home,NaN,NaN,NaN,MCI,NaN,NaN,NaN,CRY,...,26,58,2.6,66.6,1.6,0.7,2022,90.6,2.8,0.4
7,2023-03-11,Home,NaN,NaN,NaN,BRE,NaN,NaN,NaN,EVE,...,24,38,1.8,61.1,1.7,1.0,2022,73.5,2.1,0.8
8,2023-03-11,Away,NaN,NaN,NaN,TOT,NaN,NaN,NaN,NFO,...,26,45,1.8,55.4,1.5,1.1,2022,78.4,2.2,0.7
9,2023-03-11,Home,NaN,NaN,NaN,CHE,NaN,NaN,NaN,LEI,...,25,34,1.0,69.2,1.9,0.9,2022,80.6,2.0,0.4


In [16]:
if not os.getcwd().endswith('Football Forecasting Version 2'):
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

os.chdir(fr'{dir_path}\{current_season}_{current_season + 1}_Clean_Data')

today = date.today()

df_final.to_csv(fr'{current_season}_{current_season + 1}_{today.strftime("%b%d")}_matchweek_data.csv')
